# Extraction des temps d'attente

Les données que nous cherchons à comprendre sont des données issues de Mariane. Elles se présentent sous la forme d'un tableau qui présente le temps d'attente pour avoir un rendez vous chez différentes catégories de médecins pour chaque préfecture et sous-préfecture. 

Ce tableaux cependant n'est récupérable qu'en image, l'enjeu de cette extraction de donnée sera donc de convertir une image de tableau en fichier exploitable pour le traitement des données (CSV ici).

### Du document .jpg au document .csv
Pour lire le text sur un fichier image, on utilise tesseract, un logiciel de reconnaissance optique de caractère facilement utilisable depuis python.

In [ ]:
from PIL import Image
!pip install pytesseract
from pytesseract import pytesseract
import re
pytesseract.tesseract_cmd = "/work/desert-medical/tesseract.exe"

You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [ ]:
img=Image.open("/work/desert-medical/images/im1.jpg")
text = pytesseract.image_to_string(img.convert("L"))

PermissionError: [Errno 13] Permission denied: '/work/desert-medical/tesseract.exe'

In [ ]:
def completeCSV(text, fichierCSV, erreurs):
    """
    prend en entrée un des tableaux en chaine de caractère et le début du fichier CSV
    qui globalise les données des 7 tableaux.
    Complète le fichier CSV avec les donnée du tableau en entrée
    """
    liste = text.split("\n")[:-1]

    sep = '[\\\|\[\]\(\)\{\}/° "*:¢]'

    # fichierCSV = ""

    # liste_bon = []
    bon = 0
    for l in liste:
        if bon == 1 and l != ' ' and l != '':
            tmp = []
            villeAArticle = False
            for i in re.split(sep,l):
                if villeAArticle and i!='':
                    tmp[-1]+= " "+ i
                    villeAArticle=False
                else:
                    if i == "Le" or i =="La" or i == "Les" or i == "Vire":
                        villeAArticle = True
                    if i != "":
                        tmp.append(i)
            # print(tmp)
    #convertir la ligne tmp en ligne du fichier CSV
            fichierCSV += "\n" + tmp[0] + "," + tmp[1]
            for e in range(2,8):
                try:
                    if tmp[e].isdigit():
                        fichierCSV += ","+tmp[e]
                    else:
                        fichierCSV += ",NA"
                except:
                    fichierCSV += ",NA"
            if len(tmp)!=8 :
                erreurs.append(tmp[1])


            # for e in tmp[2:]:
            #     if e.isdigit():
            #         fichierCSV += ","+e
            #     else:
            #         fichierCSV += ",NA"
            # for e in range(8-len(tmp)):
            #     fichierCSV += ",NA"

            # liste_bon.append(tmp)
        else :
            # if l == 'Moyenne nationale | 98 | 140 | 21 | 101 | 158 | 74':
            if 'Moyenne nationale' in l:
                bon = 1
    return fichierCSV, erreurs


pytesseract.tesseract_cmd = "C:\\Program Files\\Tesseract-OCR\\tesseract.exe"
fichierCSV = "Rang sur 314,Territoire,Dentiste,Dermatologue,Généraliste,Gynécologue,Ophtalmologue,Pédiatre"

erreurs = []

for i in range(7):
    img=Image.open(r"C:\Users\elita\Documents\projet stat\images\im"+str(i+1)+".jpg")



    text = pytesseract.image_to_string(img.convert("L"))

    fichierCSV, erreurs = completeCSV(text, fichierCSV, erreurs)


fichier = open(r"C:\Users\elita\Documents\projet stat\donnee.csv","w")
fichier.write(fichierCSV)
fichier.close()

### Associer les villes à leur code INSEE

Pour permettre de travailler avec les atres données, on associe à chaque ville son code INSEE. Pour se faire on crée un dictionnaire qui lie chaque ville à son code INSEE à l'aide de la page wikipedia qui liste toutes les sous prefectures de france et de la page wikipedia qui liste toutes les prefectures de France.

In [ ]:
import urllib
from bs4 import BeautifulSoup
import pandas
from urllib import request
import json
from tqdm import tqdm

##
url = "https://fr.wikipedia.org/wiki/Liste_des_sous-pr%C3%A9fectures_de_France"

request_text = request.urlopen(url).read()

page = BeautifulSoup(request_text, "lxml")

lignes = page.find('table').find_all('tr')

dico = dict()
#liste=[]

for ligne in lignes[1:] :
    nouveau_dep = ligne.find('th')!=None
    cols = ligne.find_all('td')

    if nouveau_dep:
        if cols[2].text.strip() != '' :
            dico[cols[2].text.strip()]=cols[1].text.strip()
            #liste.append([cols[2].text.strip(), cols[1].text.strip()])
    else :
        dico[cols[1].text.strip()]=cols[0].text.strip()
        #liste.append([cols[1].text.strip(), cols[0].text.strip()])



##
url = "https://fr.wikipedia.org/wiki/Liste_des_pr%C3%A9fectures_de_France"

request_text = request.urlopen(url).read()

page = BeautifulSoup(request_text, "lxml")

lignes = page.find('table').find_all('tr')
# dico = dict()
# liste = []

for ligne in tqdm(lignes[1:97]):
    nom = ligne.find_all('td')[1].text.strip()

    lien = "https://fr.wikipedia.org" + ligne.find_all("a")[1].get('href')
    page_pref = BeautifulSoup(request.urlopen(lien).read(), "lxml")

    infos = page_pref.find('table').find_all('tr')
    i=0

    try :
        test = infos[i].find('th').text.strip()
    except :
        test = None
    while test != "Code commune":
        # print(infos[i].text.strip().split("\n")[0])
        i+=1
        try :
            test = infos[i].find('th').text.strip()
        except :
            test = None
    num = infos[i].find('td').text.strip()

    num = num.split(" ")[0]
    dico[nom]=num
    #liste.append([nom,num])


# Il y a aussi deux ville dans le tableau de Mariane qui ne sont pas des (sous-)préfectures
dico['Ancenis-St-Géréon']='44003'
dico['Sainte-Menehould']='51507'

# Et Paris et mal encodé car il est nomé "Paris (C)" sur wikipédia
dico['Paris'] = dico.pop('Paris (C)')

fichier = open("/work/desert-medical/data/(sous) prefectures.json","w")
json.dump(dico,fichier)
fichier.close()

100%|██████████| 96/96 [02:52<00:00,  1.79s/it]


In [ ]:
import json
fichier_dico = open("/work/desert-medical/data/(sous) prefectures.json","r")
dico = json.load(fichier_dico)
fichier_dico.close()

fichier = open("/work/desert-medical/data/donnee temps attente.csv", "r")
doc = fichier.read()
fichier.close()

#!pip install unidecode
from unidecode import unidecode

# dans la lecture des images il y a de nombreux problèmes d'accents alors on 
#utilise le module unidecode pour retirer les accents
dico1 = dict()
for i in dico :
    dico1[unidecode(i)]=dico[i]

# le dictionnaire inversé qui associe à un numéro INSEE une ville (bien orthographiée)
dico_inv = {v:k for k,v in dico.items()} 

tab=[el.split(",") for el in doc.split("\n")[1:]]

# le fichier qui contiendra la liste des temps d'attente avec le numero INSEE
nouveau_CSV="Numéro INSEE,Territoire,Dentiste,Dermatologue,Généraliste,Gynécologue,Ophtalmologue,Pédiatre,Rang sur 314"
erreurs=[]

for el in tab:
    try : # si la lecture de l'image a bien fonctinné, il arrive à associer à la ville un numéro INSEE
        num = dico1[unidecode(el[1])]
        nouveau_CSV += "\n"+num + "," + dico_inv[num]+"," + el[2]+","+el[3]+","+el[4]+","+el[5]+","+el[6]+","+el[7]+","+el[0]

    except: # sinon on le fera à la main
        erreurs.append(el[1])

print(erreurs) # il n'y a que 13 erreurs, on peut les traiter à la main

[]


In [ ]:
#"""Alençon, Ancenis-St-Géréon : 44003, Besançon, Chalon sur Saône, Florac Trois Rivières
#Le Blanc, Montmorillon, Paris, Rambouillet, Sainte-Menehould : 51507, Segré-en-Anjou Bleu, 
#Tournon-sur-Rhône, Vendôme"""

In [ ]:
fichier = open("/work/desert-medical/data/donnee temps attente (avec numero INSEE).csv", "w")
fichier.write(nouveau_CSV)
fichier.close()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=2f812232-8ed1-4e89-8f48-597e10058637' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>